In [62]:
import sys
import os
import numpy as np
import pickle
import pandas as pd
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
import itertools
import qinfer as qi
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib as mpl

from lfig import LatexFigure

sys.path.append("/home/bf16951/QMD")
import qmla

%matplotlib inline

In [1]:
results_paths = ["Dec_09/12_29", "Dec_09/12_31", "Dec_09/12_33", "Dec_09/12_32"]
best_instances = {
    "Dec_09/12_29" : 2, 
    "Dec_09/12_31" : 1, 
    "Dec_09/12_32" : 2,
    "Dec_09/12_33" : 2 
}

timing_list = [
    4000, 770, 16000, 690
]

lf = LatexFigure(
    gridspec_layout = (2,3),
    gridspec_params = {
        'hspace' : 0.4, 
        'wspace' : 0.1,
        'width_ratios' : [1,1,0.1]
    },
    legend_axis = (0,2),
    legend_span = ('all', 1),
    
)
lf.ax_counter = 0 

all_bf = pd.DataFrame()
# for res in results_paths:
#     results = qmla.load_results(
#         results_folder = "/home/bf16951/bc_results",
#         results_time = res,
#         instance_id = instance
#     )
#     qmla_instance = results['qmla_instance']
#     all_bf = all_bf.append(qmla_instance.bayes_factors_df, ignore_index=True)
    

# min_log_bf = all_bf.log10_bayes_factor.min()
# max_log_bf = all_bf.log10_bayes_factor.max()
min_log_bf = -50
max_log_bf = 50
    

for res in results_paths:
    ax = lf.new_axis()
    results = qmla.load_results(
        results_folder = "/home/bf16951/thesis/qmla_run_data",
        results_time = res,
        instance_id = best_instances[res]
    )
    qmla_instance = results['qmla_instance']

    bayes_factor_by_fscore = pd.pivot_table(
        qmla_instance.bayes_factors_df,
        values='log10_bayes_factor',
        index=['f_score_a'],
        columns=['f_score_b'],
        aggfunc=np.median
    )
    mask = np.tri(bayes_factor_by_fscore.shape[0], k=-1).T
    sns.heatmap(
        bayes_factor_by_fscore,
        cmap=qmla_instance.exploration_class.bf_cmap, 
        cbar=False, 
        vmin=min_log_bf, 
        vmax=max_log_bf, 
        mask=mask,
        annot=False,
        linewidths=.5, 
        ax = ax
    )
    ax.set_xlabel("")
    ax.set_ylabel("")

    
norm = mpl.colors.Normalize(vmin=min_log_bf,vmax=max_log_bf)
cbar = mpl.colorbar.ColorbarBase(
    lf.legend_ax, 
    cmap = qmla_instance.exploration_class.bf_cmap,
    norm = norm,
    orientation="vertical",
    label = r"$\log_{10}B_{ij}$",
)
# cbar.set_clim(0, 1.0)    
    
    
# Timings bar chart
inset_ax = inset_axes(
    lf.gridspec_axes[(0,1)], 
    width=1.1, 
    height=0.65, 
    loc='upper right'
)
inset_ax.barh(
    ['a','b','c','d'], 
    timing_list, 
    color = 'black'
)
inset_ax.semilogx()
inset_ax.set_xlim(0,20000)
inset_ax.axvline(3600, ls='--', lw=0.75, c='blue', alpha=0.5)
inset_ax.invert_yaxis()
inset_ax.set_xlabel("Time (s)")
inset_ax.xaxis.tick_top()
inset_ax.xaxis.set_label_position('top') 

lf.fig.text(0.04, 0.5, r"$f_i$", va='center', rotation="vertical")
lf.fig.text(0.5, 0.02, r"$f_j$", ha='center')   
lf.save(
    os.path.join(
        "/home/bf16951/thesis/theoretical_study/figures",
        "bayes_factors_by_f_scores.pdf"
    )
)

NameError: name 'LatexFigure' is not defined